In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')
#train_afds

In [3]:
#train_lre = train_lre.iloc[1000:2000]

In [4]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.639420,0.345684,-0.517645,-0.737002,1.313001,1.655732,0.615168,0.799339,1.648419,1.314986,...,-1.322407,-0.717348,-3.843951,-1.471274,0.945421,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.113610,0.738034,0.584856,-0.248521,0.256616,1.060159,-0.416211,0.133670,0.188327,0.805241,...,1.965741,3.525555,0.006925,1.676099,-1.116371,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.061305,-0.140605,-0.627104,-0.682908,1.337613,1.261543,-0.651291,0.307040,-0.980717,0.335530,...,-3.018432,-1.841361,0.814595,-0.200731,1.180795,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.958049,-0.608315,-0.113532,0.167272,0.911100,1.116094,-0.847214,-0.041357,-0.542335,0.199105,...,-0.548767,-1.254447,1.644647,0.555434,1.159140,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.842436,-0.417598,-0.427428,-0.485898,1.204330,1.439188,-1.056642,0.111603,-0.404313,-0.307075,...,0.297026,-1.858708,0.209784,0.583939,1.398789,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [5]:
#train_lre.groupby(['langid']).size()

In [6]:
#Choosing a single class
train_lre = train_lre.loc[train_lre['lang'] == 'ara-apc']

In [7]:
train_lre

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
2,2.061305,-0.140605,-0.627104,-0.682908,1.337613,1.261543,-0.651291,0.307040,-0.980717,0.335530,...,-3.018432,-1.841361,0.814595,-0.200731,1.180795,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.958049,-0.608315,-0.113532,0.167272,0.911100,1.116094,-0.847214,-0.041357,-0.542335,0.199105,...,-0.548767,-1.254447,1.644647,0.555434,1.159140,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.842436,-0.417598,-0.427428,-0.485898,1.204330,1.439188,-1.056642,0.111603,-0.404313,-0.307075,...,0.297026,-1.858708,0.209784,0.583939,1.398789,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
5,1.953034,-0.596223,-0.535057,-0.371161,0.592551,1.366468,-1.361346,0.299445,-0.305046,-0.537489,...,0.723645,-2.709617,1.595344,-0.267579,0.810446,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
6,1.990754,-0.430986,-0.371372,0.035754,1.007754,1.237592,-0.698143,-0.091484,-0.547547,-0.248731,...,-1.230570,-3.184290,2.312549,-1.312105,1.701691,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
7,-0.466107,-0.884151,-0.357131,-0.577324,-0.220201,1.070911,0.060307,0.799765,-0.783718,-0.765216,...,-1.309657,0.627864,-1.560878,-1.698744,-0.523284,fla_0073-b,LDC2017E22,data,ara-apc,fla_0073-b
8,-0.516523,-0.627713,-0.105311,-0.322296,-0.553714,1.011796,0.757867,1.013907,-1.236593,-0.911828,...,0.579321,-0.415504,-0.113781,-0.765633,-0.917091,fla_0073-b,LDC2017E22,data,ara-apc,fla_0073-b
9,-0.725326,-0.702411,-0.345675,-0.701975,0.130706,1.177092,0.764530,1.189533,-1.282171,-1.082967,...,-1.098577,0.045006,-0.065501,-0.826183,0.936744,fla_0073-b,LDC2017E22,data,ara-apc,fla_0073-b
10,-0.641780,-0.386634,-0.121360,-0.916782,0.394523,1.070178,1.007852,1.380711,-1.707224,-1.127637,...,0.284225,-0.634877,1.705949,0.416899,-1.166602,fla_0073-b,LDC2017E22,data,ara-apc,fla_0073-b
11,-0.578299,-0.598327,-0.278434,-0.740745,-0.022859,1.239188,0.778646,1.184929,-1.456520,-0.931703,...,-0.582347,-0.709782,-1.074811,0.894534,1.752265,fla_0073-b,LDC2017E22,data,ara-apc,fla_0073-b


In [8]:
val_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/dev_feat_BNF_h5_07Nov_Shreyas.csv')


In [9]:
#val_lre = val_lre.iloc[100:300]

In [10]:
val_lre.head()

,segmentid,language_code,data_source,speech_duration,0,1,2,3,4,5,...,491,492,493,494,495,496,497,498,499,uttid
0,lre17_ntrlosgu.sph,ara-acm,mls14,30,1.697234,0.029428,-0.400756,0.513963,-0.939232,1.500797,...,-1.314428,-0.927694,-0.370424,-0.514735,1.290885,0.688205,-0.494330,-0.053206,-1.330860,lre17_ntrlosgu
1,lre17_moxnwuqe.sph,ara-acm,mls14,10,1.648232,-0.053318,-0.562867,1.035870,-1.577741,1.593584,...,-0.929262,-1.301574,2.034934,-0.226545,-0.198926,-0.116174,0.347923,-0.870801,-2.599601,lre17_moxnwuqe
2,lre17_meesvkxz.sph,ara-acm,mls14,3,1.242829,0.675515,-0.371491,0.534970,-0.246783,0.806262,...,0.691336,0.257988,1.058771,1.018635,-1.929319,-0.307404,-0.486431,-2.839053,-2.704527,lre17_meesvkxz
3,lre17_rqmsmzui.sph,ara-acm,mls14,30,1.226681,0.014810,-0.396915,-0.097507,-0.013574,1.087025,...,1.049862,0.285627,2.385587,0.680073,1.500978,1.660566,-0.370672,-0.924109,0.096676,lre17_rqmsmzui
4,lre17_qgszpuyw.sph,ara-acm,mls14,10,1.411728,-0.119300,0.136256,0.030535,-1.029447,1.227100,...,0.155196,-1.030222,2.933880,-1.417872,-0.227513,0.748810,-0.671044,0.595977,1.722917,lre17_qgszpuyw


In [11]:
X_train = train_lre.drop(["lang",'ids','year','data','lang_id'],axis=1)
y_train = train_lre["lang"]
#y_train_uttid = train_lre["uttid"]

In [12]:
X_val = val_lre.drop(["language_code","uttid","segmentid","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid"]

In [13]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [14]:
le.classes_

array(['ara-apc'], dtype=object)

In [15]:
y_train=le.transform(y_train)

In [16]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [17]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [18]:
y_val_labels = le.transform(y_val)

In [19]:
y_val_labels[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
X_train=X_train.values
X_val=X_val.values

In [21]:
nb_classes = 14

In [22]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [23]:
Y_train = indices_to_one_hot(y_train, 14)
#Y_test = indices_to_one_hot(y_test, 14)
Y_val = indices_to_one_hot(y_val_labels, 14)

In [24]:
Y_val

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [25]:
Y_train

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [26]:
Y_val.shape

(3661, 14)

In [27]:
Y_train.shape

(22229, 14)

In [28]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (22229, 500)
22229 train samples
3661 val samples


In [29]:
X_train.shape

(22229, 500)

In [30]:
y_train.shape

(22229,)

In [31]:
#X_train,  y_train = shuffle(X_train, y_train, random_state=0)

In [32]:
X_val.shape

(3661, 500)

In [33]:
y_val_labels

array([ 0,  0,  0, ..., 13, 13, 13])

In [34]:
#X_val,  y_val_labels = shuffle(X_val, y_val_labels, random_state=0)

In [35]:
#X_train, X_test, Y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [36]:
#mnist = input_data.read_data_sets('/home/satishk/depy_04_AUG/MNIST_data', one_hot=True)
mb_size = 256
Z_dim = 100
X_dim = 500 #mnist.train.images.shape[1]
y_dim = 14 #mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3


In [37]:
X_dim, y_dim

(500, 14)

In [38]:
import torch.nn as nn

In [39]:
#Implementing Generator as Encoder-Decoder pair inspired from DAGAN base paper

#Encoder should be able to take a batch of input(of dim 500-ivector) and be able to produce its representation r

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(500, 256)
        self.fc2 = nn.Linear(256, 128)
        #self.fc3 = nn.Linear(256, 128)
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))
    
#We will use Decoder as Generator    
#Decoder should be able to take input of dim 128(r) and 100(z),concatenate r and z and 
#produce an output od dim 500-ivector    

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(228,256 )
        self.fc2 = nn.Linear(256, 500)
        #self.fc3 = nn.Linear(512,500)
    def forward(self, x):
        #inputs = torch.cat([z, x], 1)
        return F.sigmoid(self.fc2(F.relu(self.fc1(x))))

#input to Generator alias Decoder is inputs = torch.cat([z, r], 1)
#Where r is the output of encoder given x i.e., representation of x encoded by encoder


In [40]:
#As we defined the Encoder and Decoder we now Implement the Generator 

class Generator(nn.Module):
                                
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = Encoder()
        self.fc2 = Decoder()

    def forward(self, z, x):
        #inputs = torch.cat([z, x], 1)
        return self.fc2(torch.cat([z, self.fc1(x)],1))

In [41]:
#Implementation of Discriminator

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        #self.inputs = torch.cat([z, c], 1)
        self.fc1 = torch.nn.Linear(500, 128)
        self.fc2 = torch.nn.Linear(128,1)
        #self.fc3 = torch.nn.Linear(512,1)
        
    def forward(self, x):
        #inputs = torch.cat([X, c], 1)
        return F.sigmoid(self.fc2(F.relu(self.fc1(x))))


In [42]:
G = Generator()
D = Discriminator()

In [43]:
G.modules

<bound method Module.modules of Generator(
  (fc1): Encoder(
    (fc1): Linear(in_features=500, out_features=256)
    (fc2): Linear(in_features=256, out_features=128)
  )
  (fc2): Decoder(
    (fc1): Linear(in_features=228, out_features=256)
    (fc2): Linear(in_features=256, out_features=500)
  )
)>

In [44]:
ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))
ones_label_fake = Variable(torch.ones(mb_size*2))
#ones_label = ones_label.cuda()
#zeros_label = zeros_label.cuda()
#ones_label_fake = ones_label_fake.cuda()


In [45]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [46]:
# Loss function
#criterion = torch.nn.BCELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [47]:
len(X_train)

22229

In [ ]:
from torch.nn.functional import binary_cross_entropy

In [ ]:
batch_size = mb_size
# Start training
for epoch in range(50):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train) :
        

        start= it
        end= it + batch_size


        z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train[start:end]

        #c = Y_train[start:end]
        X = Variable(torch.from_numpy(X))
        #c = Variable(torch.from_numpy(c.astype('float32')))
        #z = z.cuda()
        #X = X.cuda()
        # Dicriminator forward-loss-backward-update
        G_sample = G(z, X)
        D_real = D(X)
        D_fake = D(G_sample)

        D_loss_real = binary_cross_entropy(D_real, ones_label)
        D_loss_fake = binary_cross_entropy(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake

        D_loss.backward()
        D_solver.step()

        # Housekeeping - reset gradient
        D.zero_grad()

        # Generator forward-loss-backward-update
        z = Variable(torch.randn(mb_size, Z_dim))
        G_sample = G(z, X)
        X_fake = torch.cat([G_sample, X], 0)
        D_fake = D(X_fake) #Here we need to give Xi along with Xg(i.e. Xg+X or G_sample+X)

        G_loss = binary_cross_entropy(D_fake, ones_label_fake)

        G_loss.backward()
        G_solver.step()

        # Housekeeping - reset gradient
        D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.numpy(), G_loss.data.numpy()))



        it+= batch_size
           

epoch: 0


/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Iter-0; D_loss: [ 1.39441705]; G_loss: [ 0.70605069]
Iter-256; D_loss: [ 1.33229136]; G_loss: [ 0.71103138]
Iter-512; D_loss: [ 1.30183434]; G_loss: [ 0.72898847]
Iter-768; D_loss: [ 1.28419924]; G_loss: [ 0.75008571]
Iter-1024; D_loss: [ 1.21312332]; G_loss: [ 0.74295187]
Iter-1280; D_loss: [ 1.23297703]; G_loss: [ 0.78058445]
Iter-1536; D_loss: [ 1.18391418]; G_loss: [ 0.78383082]
Iter-1792; D_loss: [ 1.15817332]; G_loss: [ 0.79684937]
Iter-2048; D_loss: [ 1.15773106]; G_loss: [ 0.82139826]
Iter-2304; D_loss: [ 1.09903848]; G_loss: [ 0.8152433]
Iter-2560; D_loss: [ 1.11218691]; G_loss: [ 0.84535325]
Iter-2816; D_loss: [ 1.09432256]; G_loss: [ 0.8587352]
Iter-3072; D_loss: [ 1.06049848]; G_loss: [ 0.86463016]
Iter-3328; D_loss: [ 1.04447532]; G_loss: [ 0.87792832]
Iter-3584; D_loss: [ 1.05843234]; G_loss: [ 0.9064737]
Iter-3840; D_loss: [ 1.04101443]; G_loss: [ 0.91912866]
Iter-4096; D_loss: [ 0.99981117]; G_loss: [ 0.92047578]
Iter-4352; D_loss: [ 0.99126458]; G_loss: [ 0.93622297]
I

Iter-15360; D_loss: [ 0.57255465]; G_loss: [ 0.97216451]
Iter-15616; D_loss: [ 0.60807288]; G_loss: [ 0.99630016]
Iter-15872; D_loss: [ 0.55788362]; G_loss: [ 0.97834599]
Iter-16128; D_loss: [ 0.59136403]; G_loss: [ 1.00114596]
Iter-16384; D_loss: [ 0.57650805]; G_loss: [ 1.00021744]
Iter-16640; D_loss: [ 0.55404449]; G_loss: [ 0.99527764]
Iter-16896; D_loss: [ 0.55322886]; G_loss: [ 1.00134122]
Iter-17152; D_loss: [ 0.5547716]; G_loss: [ 1.00856042]
Iter-17408; D_loss: [ 0.54886848]; G_loss: [ 1.01181173]
Iter-17664; D_loss: [ 0.53245211]; G_loss: [ 1.01027083]
Iter-17920; D_loss: [ 0.56657588]; G_loss: [ 1.03314185]
Iter-18176; D_loss: [ 0.55048335]; G_loss: [ 1.03137875]
Iter-18432; D_loss: [ 0.53099513]; G_loss: [ 1.02802074]
Iter-18688; D_loss: [ 0.5475347]; G_loss: [ 1.04169559]
Iter-18944; D_loss: [ 0.51789707]; G_loss: [ 1.03349006]
Iter-19200; D_loss: [ 0.53758478]; G_loss: [ 1.04894757]
Iter-19456; D_loss: [ 0.50785607]; G_loss: [ 1.04043686]
Iter-19712; D_loss: [ 0.51469183]

Iter-8704; D_loss: [ 0.24179605]; G_loss: [ 1.46466231]
Iter-8960; D_loss: [ 0.24977393]; G_loss: [ 1.47192049]
Iter-9216; D_loss: [ 0.23023611]; G_loss: [ 1.46577239]
Iter-9472; D_loss: [ 0.20916834]; G_loss: [ 1.4596324]
Iter-9728; D_loss: [ 0.21027496]; G_loss: [ 1.46401823]
Iter-9984; D_loss: [ 0.23017064]; G_loss: [ 1.47800791]
Iter-10240; D_loss: [ 0.22307223]; G_loss: [ 1.47844315]
Iter-10496; D_loss: [ 0.2250703]; G_loss: [ 1.48309243]
Iter-10752; D_loss: [ 0.22001687]; G_loss: [ 1.4844209]
Iter-11008; D_loss: [ 0.21207681]; G_loss: [ 1.48409748]
Iter-11264; D_loss: [ 0.22197349]; G_loss: [ 1.49258995]
Iter-11520; D_loss: [ 0.24205106]; G_loss: [ 1.50592101]
Iter-11776; D_loss: [ 0.23030077]; G_loss: [ 1.50371993]
Iter-12032; D_loss: [ 0.23834227]; G_loss: [ 1.51081169]
Iter-12288; D_loss: [ 0.2144959]; G_loss: [ 1.5028075]
Iter-12544; D_loss: [ 0.23052785]; G_loss: [ 1.51426208]
Iter-12800; D_loss: [ 0.21032807]; G_loss: [ 1.50785792]
Iter-13056; D_loss: [ 0.21850537]; G_loss:

Iter-2048; D_loss: [ 0.12843327]; G_loss: [ 1.82415986]
Iter-2304; D_loss: [ 0.11475236]; G_loss: [ 1.82007885]
Iter-2560; D_loss: [ 0.12485467]; G_loss: [ 1.82752228]
Iter-2816; D_loss: [ 0.12110968]; G_loss: [ 1.828269]
Iter-3072; D_loss: [ 0.12606421]; G_loss: [ 1.83295631]
Iter-3328; D_loss: [ 0.1149573]; G_loss: [ 1.8299067]
Iter-3584; D_loss: [ 0.11483937]; G_loss: [ 1.83257747]
Iter-3840; D_loss: [ 0.11733433]; G_loss: [ 1.83635175]
Iter-4096; D_loss: [ 0.12509146]; G_loss: [ 1.84257102]
Iter-4352; D_loss: [ 0.11097211]; G_loss: [ 1.83803582]
Iter-4608; D_loss: [ 0.10868907]; G_loss: [ 1.83963072]
Iter-4864; D_loss: [ 0.11722612]; G_loss: [ 1.84642351]
Iter-5120; D_loss: [ 0.11058782]; G_loss: [ 1.8456794]
Iter-5376; D_loss: [ 0.10779258]; G_loss: [ 1.84692109]
Iter-5632; D_loss: [ 0.10426028]; G_loss: [ 1.84783924]
Iter-5888; D_loss: [ 0.10148149]; G_loss: [ 1.84926975]
Iter-6144; D_loss: [ 0.1080658]; G_loss: [ 1.85501754]
Iter-6400; D_loss: [ 0.11663115]; G_loss: [ 1.86136413

In [ ]:
type(D_real)

In [ ]:
type(X)

In [ ]:
df_SF=pd.DataFrame()
samples_per_class = 1000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(14):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    
    

In [ ]:
c_df.shape

In [ ]:
df_SF.shape[0]

In [ ]:
c_gen = df_SF.values

In [ ]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [ ]:
z_gen.data.shape

In [ ]:
c_gen = Variable(torch.from_numpy(c_gen))

In [ ]:
samples = G(z_gen, c_gen)

In [ ]:
samples.data.shape

In [ ]:
#Taking the generated iVectors we will try to check the acc by MLP

In [ ]:
train_X1 = samples.data.numpy()

In [ ]:
type(train_X1)

In [ ]:
train_y1 = c_gen.data.numpy()

In [ ]:
type(train_y1)

In [ ]:
train_y1

In [ ]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)


In [ ]:
train_X1 = pd.DataFrame(train_X1)
train_y1 = pd.DataFrame(train_y1)

In [ ]:
#train_y

In [ ]:
train_X = X_train.append(train_X1, ignore_index=True)
train_y = Y_train.append(train_y1, ignore_index=True)

In [ ]:
#train_X,  train_y = shuffle(train_X, train_y, random_state=0)

In [ ]:
train_X = train_X.values
train_y = train_y.values

In [ ]:
train_y

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [ ]:
batch_size = 256
nb_epoch=30

In [ ]:
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X, train_y, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_test , y_test),callbacks=[saveBestModel])

In [ ]:
#Frame label accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('ERROR after Data Augmentation %:', 1-score[1])